# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os

print(os.uname())

# In the cloud environment?
if 'root' in os.environ['HOME']:
    UENVPATH = '/data/'
    !pip -q install /home/workspace/python

# In the standalone environment?
if 'ferenc' in os.environ['HOME']:
    UENVPATH = '/home/ferenc/Python/rl/udadrl/data/'


from unityagents import UnityEnvironment
import numpy as np

import pprint as pp

posix.uname_result(sysname='Linux', nodename='dcebd75ec284', release='4.15.0-1080-gcp', version='#90~16.04.1-Ubuntu SMP Fri Jul 10 19:11:10 UTC 2020', machine='x86_64')
tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.7 which is incompatible.


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
# ONE Agent, Standalone
#UENVCHOICE = 'Reacher_Linux/Reacher.x86_64'

# TWENTY Agents, Standalone
#UENVCHOICE = 'Reacher_Linux_20/Reacher.x86_64'

# ONE Agent, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'

# TWENTY Agents, Cloud, No-Visuals 
UENVCHOICE = 'Reacher_Linux_NoVis/Reacher.x86_64'

env = UnityEnvironment( file_name=os.path.join( UENVPATH, UENVCHOICE ) )




INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# FA DEBUG! DEBUG! DEBUG! 
type(env)

unityagents.environment.UnityEnvironment

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [5]:
# FA  DEBUG! DEBUG! DEBUG! 
print(env.brain_names)
print( type(brain) )
brain.brain_name

['ReacherBrain']
<class 'unityagents.brain.BrainParameters'>


'ReacherBrain'

In [6]:
# FA  DEBUG! DEBUG! DEBUG!
print(f'Observation Space: {brain.vector_observation_space_size} dimensions of {brain.vector_observation_space_type} type') 
print(f'Action Space: {brain.vector_action_space_size} dimensions of {brain.vector_observation_space_type} type')

Observation Space: 33 dimensions of continuous type
Action Space: 4 dimensions of continuous type


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [8]:
# Should a run with random actions be perfomed first?
RANDOMRUN = False

# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

while RANDOMRUN == True:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        #print(f'\r#{epc} Rewards = {rewards}')
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
                #pp.pprint(list(action))
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents
#
Total score (averaged over agents) this episode: 0.0


### When finished, you can close the environment.
#### Just not for now because unit testing below

In [9]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

##### FA: One BIG MISUNDERSTANDING & STACKS OF TENSORS (23-08-2020) --> #FA; BMSoT:
It has cost me several days if not weeks to get behind the fact that the [A2C sample implementation of Miguel](https://github.com/mimoralea/gdrl/blob/master/notebooks/chapter_11/chapter-11.ipynb) is working with **stacks of tensors** instead of single tensors. Which was especially hard to find because the PyTorch code looks exactly the same for both.

In the end, when I thought about it, it makes sense and is a nifty feature of PyTorch. It is just not obvious to people like me, without in deep insights in the inner workings of PyTorch. 

Furthermore the authors of the PyTorch documentation seem not to make it too visible, I had to dig it out of one of the function definitions I use, however inderectly over the layer defintion for a2cnet:

https://pytorch.org/docs/0.4.0/_modules/torch/nn/functional.html#linear

> def linear(input, weight, bias=None):
>    """
>    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.
>
>    Shape:
>        - Input: :math:`(N, *, in\_features)` where `*` means any number of
>          additional dimensions
>        - Weight: :math:`(out\_features, in\_features)`
>        - Bias: :math:`(out\_features)`
>        - Output: :math:`(N, *, out\_features)`
>    """
>    if input.dim() == 2 and bias is not None:
>        # fused op is marginally faster
>        return torch.addmm(bias, input, weight.t())
>
>    output = input.matmul(weight.t())
>    if bias is not None:
>        output += bias
>    return output


#### Tracing the error in the 'mya2cnet' module code refactoring below

In [10]:
# TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! 
#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(20200808) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

# Format: IN_Num [Layer 1] (OUT_Num = IN_Num) [Layer 2] OUT_Num = ...
HIDDEN_DIMS_DEFAULT = {
    'shared' : (512, 512, 256, 256),
    'actor' : (256, 128, 128, 64),
    'critic' : (256, 128, 128, 64)
}
hidden_dims = HIDDEN_DIMS_DEFAULT

hlayers = dict()

hlayers['shared'] = nn.ModuleList()
hlayers['actor'] = nn.ModuleList()
hlayers['critic'] = nn.ModuleList()




In [11]:
# Input layer
input_layer = nn.Linear( 33, hidden_dims['shared'][0] )

In [12]:
input_layer

Linear(in_features=33, out_features=512, bias=True)

In [13]:
# Hidden layers shared
for i in range( len(hidden_dims['shared']) -1 ):
    hlayers['shared'].append( nn.Linear( hidden_dims['shared'][i], hidden_dims['shared'][i+1] ) )

In [14]:
hlayers['shared']

ModuleList(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=256, bias=True)
  (2): Linear(in_features=256, out_features=256, bias=True)
)

In [15]:
# Actor layers
for i in range( len(hidden_dims['actor']) ):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    if i == 0:
        hlayers['actor'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['actor'][i] ) )
    else:
        # hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i], hidden_dims['actor'][i+1] ERROR !!!
        hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i-1], hidden_dims['actor'][i] ) )
    #print( i, hlayers['actor'] ) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
        
actor_out_layer = nn.Linear( hidden_dims['actor'][-1], 4 )

In [16]:
hlayers['actor']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [17]:
actor_out_layer

Linear(in_features=64, out_features=4, bias=True)

In [18]:
#Critic layers
for i in range( len(hidden_dims['critic']) ):
    if i == 0:
        hlayers['critic'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['critic'][i] ) )
    else:
        hlayers['critic'].append( nn.Linear( hidden_dims['critic'][i-1], hidden_dims['critic'][i] ) )
critic_out_layer = nn.Linear( hidden_dims['critic'][-1], 1 )

In [19]:
hlayers['critic']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [20]:
critic_out_layer

Linear(in_features=64, out_features=1, bias=True)

In [21]:
# Prevents non Pytorch Tensor Object entering the processing stream
def torch_format(state):
    x = state
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
        if len(x.size()) == 1:
            x = x.unsqueeze(0)
    return x

In [22]:
def forward(state):
    check_tensor = lambda x: isinstance(x, torch.Tensor)
    x_act = True 
    x_crit = True

    x = torch_format(state)
    x = F.relu(  input_layer(x) )
    for label in ['shared', 'actor', 'critic']:
        for hlayer in  hlayers[label]:
            if label == 'shared':
                x = F.relu(  hlayer(x) )
            if label == 'actor':
                x_act = F.relu(  hlayer(x_act) )
            if label == 'critic':
                x_crit = F.relu(  hlayer(x_crit) )

        # Thx2: https://discuss.pytorch.org/t/copy-deepcopy-vs-clone/55022
        if ( type(x_act) == bool ):
            x_act = x.clone()  # Create an Inplace copy...
        if ( type(x_crit) == bool ):
            x_crit = x.clone() # ...after processing shared layers

    return  actor_out_layer(x_act),  critic_out_layer(x_crit) 

#### Test if A STACK OF (BMSoT) states are propagated through the debug network
And make a list of outputs of two A2C instances

In [23]:
#al = []
#bl = []

#for (dc, st) in enumerate(states):
#    al.append(a)
#    bl.append(b)

#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!    
    
#FA; BMSoT: No need to iterate through states any more!
a,b = forward(states)
print(f'Actor: {a}')
print(f'Critic: {b}')

Actor: tensor(1.00000e-02 *
       [[ 4.6719,  4.9989, -4.4549,  4.1292],
        [ 4.6650,  4.9899, -4.4703,  4.1551],
        [ 4.6869,  4.9992, -4.3600,  4.1201],
        [ 4.7230,  4.9857, -4.4225,  4.1159],
        [ 4.6677,  4.9947, -4.4496,  4.1475],
        [ 4.6942,  4.9968, -4.3605,  4.1147],
        [ 4.6999,  4.9755, -4.4776,  4.2808],
        [ 4.7155,  4.9931, -4.4209,  4.1052],
        [ 4.7037,  4.9776, -4.4285,  4.1029],
        [ 4.7089,  5.0017, -4.4363,  4.2566],
        [ 4.6913,  4.9975, -4.3603,  4.1124],
        [ 4.7112,  4.9769, -4.4726,  4.2806],
        [ 4.7113,  4.9813, -4.4270,  4.1114],
        [ 4.6580,  4.9798, -4.4859,  4.1956],
        [ 4.7024,  4.9786, -4.4259,  4.1021],
        [ 4.7159,  5.0058, -4.4159,  4.2178],
        [ 4.7071,  4.9804, -4.4285,  4.1044],
        [ 4.7252,  4.9830, -4.4250,  4.1183],
        [ 4.6802,  4.9738, -4.4947,  4.2566],
        [ 4.6776,  4.9950, -4.4435,  4.1223]])
Critic: tensor(1.00000e-02 *
       [[ 4.8230],
   

#### Test if A STACK OF (BMSoT) states is propagated through the imported network

In [24]:
#from mya2cnet import A2CNetwork
import mya2cnet

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cnet)

tstnet1 = mya2cnet.A2CNetwork(33, 4) #FA; BMSoT: Looks exactly the same...
tstnet2 = mya2cnet.A2CNetwork(33, 4) #FA; BMSoT: ...like without stacks!

In [25]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    
a1,b1 = tstnet1.forward(states)
a2,b2 = tstnet2.forward(states)

print(f'Dist. Actor stacks 1-2: {torch.dist(a1, a2)}'.ljust(50) + ' # ' +\
     f'Dist. Actor 1 stacks - Notebook stacks {torch.dist(a1, a)}'.rjust(50))
print(f'Dist. Critic stacks 1-2: {torch.dist(b1, b2)}'.ljust(50) + ' # ' +\
     f'Dist. Critic 1 stacks - Notebook stacks {torch.dist(b1, b)}'.rjust(50))

Dist. Actor stacks 1-2: 0.0                        #         Dist. Actor 1 stacks - Notebook stacks 0.0
Dist. Critic stacks 1-2: 0.0                       #        Dist. Critic 1 stacks - Notebook stacks 0.0


In [26]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):
#    print(f'({dc+1}) -> {tstnet1.fullpass(st)}')
    
tstnet1.fullpass(states)

(tensor(1.00000e-02 *
        [[ 4.6719,  4.9989, -4.4549,  4.1292],
         [ 4.6650,  4.9899, -4.4703,  4.1551],
         [ 4.6869,  4.9992, -4.3600,  4.1201],
         [ 4.7230,  4.9857, -4.4225,  4.1159],
         [ 4.6677,  4.9947, -4.4496,  4.1475],
         [ 4.6942,  4.9968, -4.3605,  4.1147],
         [ 4.6999,  4.9755, -4.4776,  4.2808],
         [ 4.7155,  4.9931, -4.4209,  4.1052],
         [ 4.7037,  4.9776, -4.4285,  4.1029],
         [ 4.7089,  5.0017, -4.4363,  4.2566],
         [ 4.6913,  4.9975, -4.3603,  4.1124],
         [ 4.7112,  4.9769, -4.4726,  4.2806],
         [ 4.7113,  4.9813, -4.4270,  4.1114],
         [ 4.6580,  4.9798, -4.4859,  4.1956],
         [ 4.7024,  4.9786, -4.4259,  4.1021],
         [ 4.7159,  5.0058, -4.4159,  4.2178],
         [ 4.7071,  4.9804, -4.4285,  4.1044],
         [ 4.7252,  4.9830, -4.4250,  4.1183],
         [ 4.6802,  4.9738, -4.4947,  4.2566],
         [ 4.6776,  4.9950, -4.4435,  4.1223]]), tensor(1.00000e-02 *
        [[ 4.82

In [27]:
#FA; BMSoT: OBSOLETE
#for (dc, st) in enumerate(states):
#    print(f'({dc+1})-> {tstnet1.select_action(st)}') #, end=' ')
    
tstnet1.select_action(states)

tensor(1.00000e-02 *
       [[ 4.6719,  4.9989, -4.4549,  4.1292],
        [ 4.6650,  4.9899, -4.4703,  4.1551],
        [ 4.6869,  4.9992, -4.3600,  4.1201],
        [ 4.7230,  4.9857, -4.4225,  4.1159],
        [ 4.6677,  4.9947, -4.4496,  4.1475],
        [ 4.6942,  4.9968, -4.3605,  4.1147],
        [ 4.6999,  4.9755, -4.4776,  4.2808],
        [ 4.7155,  4.9931, -4.4209,  4.1052],
        [ 4.7037,  4.9776, -4.4285,  4.1029],
        [ 4.7089,  5.0017, -4.4363,  4.2566],
        [ 4.6913,  4.9975, -4.3603,  4.1124],
        [ 4.7112,  4.9769, -4.4726,  4.2806],
        [ 4.7113,  4.9813, -4.4270,  4.1114],
        [ 4.6580,  4.9798, -4.4859,  4.1956],
        [ 4.7024,  4.9786, -4.4259,  4.1021],
        [ 4.7159,  5.0058, -4.4159,  4.2178],
        [ 4.7071,  4.9804, -4.4285,  4.1044],
        [ 4.7252,  4.9830, -4.4250,  4.1183],
        [ 4.6802,  4.9738, -4.4947,  4.2566],
        [ 4.6776,  4.9950, -4.4435,  4.1223]])

#### Test the agent instance

In [28]:
import mya2cagent

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cagent)

agent = mya2cagent.a2cagent(len(env_info.agents), env, brain, max_steps = 250)

In [29]:
procs = mya2cagent.MultiProcEnv(env, 20200808, len(env_info.agents))

In [30]:
agent.train(states)

Training epoch: 250 #

In [31]:
# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)


import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 

while True:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = agent.a2c_net.select_action(states)
    actions = np.clip(actions.detach().numpy(), -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        #print(f'\r#{epc} Rewards = {rewards}')
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
                #pp.pprint(list(action))
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

--Return--
> <ipython-input-31-79f5e1d9b852>(11)<module>()->None
-> import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug!
(Pdb) c
#    17  -> Agent 06 got reward +0.01000 for action: [-0.12462758 -0.52394634  0.20811877  0.51974171]
#    18  -> Agent 06 got reward +0.04000 for action: [-0.12383676 -0.52451926  0.20735133  0.52142119]
#    19  -> Agent 06 got reward +0.04000 for action: [-0.12326364 -0.52439469  0.20663738  0.52200472]
#    20  -> Agent 06 got reward +0.04000 for action: [-0.12688141 -0.52740318  0.20856172  0.52278829]
#    21  -> Agent 06 got reward +0.04000 for action: [-0.13372868 -0.54025328  0.2157031   0.52929348]
#    22  -> Agent 06 got reward +0.04000 for action: [-0.13537787 -0.54243207  0.21628863  0.53126538]
#    23  -> Agent 06 got reward +0.04000 for action: [-0.13609277 -0.54385829  0.21659181  0.53285706]
#    24  -> Agent 06 got reward +0.04000 for action: [-0.13670292 -0.54519916  0.21700078  0.53407121]
#    25  -> Agent 06 got rew

#   234  -> Agent 10 got reward +0.04000 for action: [-0.13610397 -0.60734588  0.23008028  0.58879685]
#   235  -> Agent 10 got reward +0.04000 for action: [-0.13657607 -0.60562301  0.22991818  0.58720428]
#   236  -> Agent 10 got reward +0.01000 for action: [-0.13687156 -0.6028766   0.22947478  0.58460259]
#   241  -> Agent 18 got reward +0.04000 for action: [-0.14414769 -0.56790572  0.22355619  0.5521394 ]
#   242  -> Agent 18 got reward +0.04000 for action: [-0.1436795  -0.5770731   0.22499996  0.56103176]
#   243  -> Agent 18 got reward +0.04000 for action: [-0.14199102 -0.58934999  0.22775888  0.57356906]
#   244  -> Agent 18 got reward +0.02000 for action: [-0.14144847 -0.59538549  0.22914654  0.58051622]
#   248  -> Agent 02 got reward +0.01000 for action: [-0.13364281 -0.59905761  0.2305018   0.58466303]
#   249  -> Agent 02 got reward +0.04000 for action: [-0.13093387 -0.60678631  0.23239982  0.59274673]
#   250  -> Agent 02 got reward +0.04000 for action: [-0.13346332 -0.6027

#   426  -> Agent 12 got reward +0.04000 for action: [-0.13880567 -0.59719342  0.22825035  0.5811044 ]
#   427  -> Agent 12 got reward +0.04000 for action: [-0.13774827 -0.60184872  0.22924674  0.58487904]
#   428  -> Agent 12 got reward +0.04000 for action: [-0.13860071 -0.57117969  0.22298726  0.554371  ]
#   429  -> Agent 12 got reward +0.03000 for action: [-0.13239938 -0.55203164  0.21562502  0.53907186]
#   460  -> Agent 11 got reward +0.02000 for action: [-0.14315236 -0.5559743   0.22190002  0.54344714]
#   461  -> Agent 11 got reward +0.04000 for action: [-0.14396143 -0.55886251  0.22219452  0.54628086]
#   477  -> Agent 15 got reward +0.01000 for action: [-0.13420951 -0.53179848  0.21555918  0.52040958]
#   478  -> Agent 15 got reward +0.04000 for action: [-0.13449579 -0.53404707  0.2162818   0.52212989]
#   479  -> Agent 15 got reward +0.04000 for action: [-0.13529994 -0.53710151  0.21729681  0.52410042]
#   480  -> Agent 15 got reward +0.04000 for action: [-0.1372195  -0.5450

#   626  -> Agent 15 got reward +0.03000 for action: [-0.13714543 -0.54045308  0.21739024  0.52831721]
#   627  -> Agent 15 got reward +0.04000 for action: [-0.13685007 -0.53980696  0.21705863  0.52779675]
#   628  -> Agent 15 got reward +0.04000 for action: [-0.1343219  -0.53258902  0.21434015  0.5234701 ]
#   629  -> Agent 15 got reward +0.04000 for action: [-0.1333482  -0.53185952  0.21368289  0.52386439]
#   630  -> Agent 15 got reward +0.01000 for action: [-0.13323797 -0.532453    0.21357685  0.52456844]
#   636  -> Agent 03 got reward +0.01000 for action: [-0.13995486 -0.54185528  0.21762902  0.53195393]
#   637  -> Agent 03 got reward +0.04000 for action: [-0.13969843 -0.54268426  0.21770185  0.53317261]
#   638  -> Agent 03 got reward +0.04000 for action: [-0.13898197 -0.54260683  0.21780631  0.53426707]
#   639  -> Agent 03 got reward +0.02000 for action: [-0.1378729  -0.54091972  0.2169818   0.53366303]
#   641  -> Agent 13 got reward +0.04000 for action: [-0.14074668 -0.5879

#   783  -> Agent 03 got reward +0.04000 for action: [-0.14568417 -0.58390188  0.22530887  0.56586087]
#     &  -> Agent 15 got reward +0.04000 for action: [-0.13253856 -0.53469986  0.21549481  0.526788  ]
#   784  -> Agent 03 got reward +0.04000 for action: [-0.14608523 -0.58277535  0.22545144  0.56468421]
#     &  -> Agent 15 got reward +0.04000 for action: [-0.13258994 -0.53556025  0.21615189  0.5278995 ]
#   785  -> Agent 15 got reward +0.03000 for action: [-0.13264148 -0.53341788  0.21620971  0.52724648]
#   786  -> Agent 16 got reward +0.01000 for action: [-0.13885692 -0.60143137  0.22916755  0.58140647]
#   787  -> Agent 16 got reward +0.04000 for action: [-0.13862145 -0.6028623   0.22946477  0.58290505]
#   788  -> Agent 16 got reward +0.04000 for action: [-0.1383052  -0.60384518  0.22965276  0.58411425]
#   789  -> Agent 16 got reward +0.04000 for action: [-0.13820434 -0.60435534  0.22969767  0.5847165 ]
#   790  -> Agent 16 got reward +0.03000 for action: [-0.13815513 -0.6038

#   909  -> Agent 09 got reward +0.03000 for action: [-0.14200631 -0.56776035  0.22106668  0.54900157]
#   918  -> Agent 03 got reward +0.01000 for action: [-0.13983192 -0.56149894  0.221212    0.54645574]
#   919  -> Agent 03 got reward +0.04000 for action: [-0.13997236 -0.56526643  0.22131383  0.54895908]
#   920  -> Agent 03 got reward +0.04000 for action: [-0.13893713 -0.57525301  0.22227514  0.55737537]
#   921  -> Agent 03 got reward +0.02000 for action: [-0.13702163 -0.58748925  0.22473019  0.56833506]
#   990  -> Agent 19 got reward +0.02000 for action: [-0.12251455 -0.52324635  0.2065354   0.51867294]
#   991  -> Agent 19 got reward +0.04000 for action: [-0.12118733 -0.52298993  0.20511088  0.51915145]
#   999  -> Agent 19 got reward +0.01000 for action: [-0.14874028 -0.60724819  0.2321254   0.5938319 ]
#  1000  -> Agent 19 got reward +0.04000 for action: [-0.14617752 -0.60357821  0.23068705  0.59076625]
#  1001  -> Agent 19 got reward +0.04000 for action: [-0.14818606 -0.6060

In [32]:
env.close()